# TensorTools Runtime Demo

Demonstrates the usage of the runtime using a simple autoencoder model.

In [ ]:
# Force matplotlib to use inline rendering
%matplotlib inline

import os
import sys

# add path to libraries for ipython
sys.path.append(os.path.expanduser("~/libs"))

import numpy as np
import tensorflow as tf
import tensortools as tt

In [ ]:
TRAIN_DIR = "/work/sauterme/train-examples/runtime-demo"

BATCH_SIZE = 24
REG_LAMBDA = 5e-4
INITIAL_LR = 0.001
LR_DECAY_STEP_INTERVAL = 10000
LR_DECAY_RATE = 0.5
NUM_GPUS = 2

In [ ]:
DATA_ROOT = "/work/sauterme/data"
dataset_train = tt.datasets.mnist.MNISTTrainDataset(DATA_ROOT)
dataset_valid = tt.datasets.mnist.MNISTValidDataset(DATA_ROOT)
dataset_test = tt.datasets.mnist.MNISTTestDataset(DATA_ROOT)

In [ ]:
class SimpleAutoencoderModel(tt.model.AbstractModel):    
    def __init__(self, reg_lambda=0.0):
        super(SimpleAutoencoderModel, self).__init__(reg_lambda)
        
    @tt.utils.attr.override
    def inference(self, inputs, targets, feeds,
                  is_training, device_scope, memory_device):
        x = tf.contrib.layers.flatten(inputs)
        encoded = tt.network.fc("FC_Enc", x, 64,
                                weight_init=tf.contrib.layers.xavier_initializer(),
                                bias_init=0.0,
                                regularizer=tf.contrib.layers.regularizers.l2_regularizer(self.reg_lambda),
                                activation=tf.nn.relu,
                                device=memory_device)
        representation = encoded
        decoded = tt.network.fc("FC_Dec", representation, x.get_shape()[1],
                                weight_init=tf.contrib.layers.xavier_initializer(),
                                bias_init=0.0,
                                regularizer=tf.contrib.layers.regularizers.l2_regularizer(self.reg_lambda),
                                activation=tf.nn.sigmoid,
                                device=memory_device)

        return tf.reshape(decoded, [-1] + targets.get_shape().as_list()[1:])
    
    @tt.utils.attr.override
    def loss(self, predictions, targets, device_scope):
        loss1 = tt.loss.mse(predictions, targets)
        loss2 = tt.loss.bce(predictions, targets)
        
        tf.add_to_collection(tt.core.LOG_LOSSES, loss1)
        tf.add_to_collection(tt.core.LOG_LOSSES, loss2)
        
        return tf.add(0.5 * loss1, 0.5 * loss2, name="25mse_75bce")
    
    @tt.utils.attr.override
    def evaluation(self, predictions, targets, device_scope=None):
        psnr = tt.image.psnr(predictions, targets)
        sharpdiff = tt.image.sharp_diff(predictions, targets)
        ssim = tt.image.ssim(predictions, targets, L=1.0)
        
        return {"psnr": psnr, "sharpdiff": sharpdiff, "ssim": ssim}

In [ ]:
tt.hardware.set_cuda_devices([2])
runtime = tt.core.DefaultRuntime(train_dir=TRAIN_DIR)
#runtime = tt.core.MultiGpuRuntime(NUM_GPUS, train_dir=TRAIN_DIR)
runtime.register_datasets(dataset_train, dataset_valid, dataset_test)
runtime.register_model(SimpleAutoencoderModel(reg_lambda=REG_LAMBDA))
optimizer = tt.training.Optimizer('adam', INITIAL_LR)
optimizer.set_decay(LR_DECAY_STEP_INTERVAL, LR_DECAY_RATE)
runtime.register_optimizer(optimizer)
runtime.build(is_autoencoder=True)

In [ ]:
def on_valid(rt, gstep):
    print ("On-Validate Hook...")

In [ ]:
runtime.train(BATCH_SIZE, steps=1000, on_validate=on_valid,
              display_steps=25, do_summary=False, do_checkpoints=False)

In [ ]:
runtime.validate(batch_size=50)

In [ ]:
runtime.test(batch_size=50)

#### Use random input of same size as in training:

In [ ]:
image_shape = dataset_train.input_shape
if len(image_shape) == 4: # MovingMNIST / UCF11
    fake_input_batch = np.random.rand(4,1,image_shape[-3],image_shape[-2],image_shape[-1])
else: # MNIST
    fake_input_batch = np.random.rand(4,image_shape[-3],image_shape[-2],image_shape[-1])
    
print(fake_input_batch.shape)
predictions = runtime.predict(fake_input_batch)

print(predictions.shape)

print(fake_input_batch.min(), fake_input_batch.max())
print(predictions.min(), predictions.max())

tt.visualization.display_batch(fake_input_batch, title="Inputs")
tt.visualization.display_batch(predictions, title="Predictions")

#### Use random image bigger than in training:

In [ ]:
SIZE_FACTOR = 2.0
image_shape = dataset_train.input_shape
if len(image_shape) == 4: # MovingMNIST / UCF11
    fake_input_batch = np.random.rand(4,1,
                                      int(image_shape[-3] * SIZE_FACTOR),
                                      int(image_shape[-2] * SIZE_FACTOR),
                                      image_shape[-1])
else: # MNIST
    fake_input_batch = np.random.rand(4,
                                      int(image_shape[-3] * SIZE_FACTOR),
                                      int(image_shape[-2] * SIZE_FACTOR),
                                      image_shape[-1])
    
print(fake_input_batch.shape)
predictions = runtime.predict(fake_input_batch)

print(predictions.shape)

print(fake_input_batch.min(), fake_input_batch.max())
print(predictions.min(), predictions.max())

tt.visualization.display_batch(fake_input_batch, title="Inputs")
tt.visualization.display_batch(predictions, title="Predictions")

In [ ]:
runtime.close()